<a href="https://colab.research.google.com/github/federicoweill/pycardano-tutorial/blob/main/pycardano_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Jupyter Notebook for Cardano on Demeter

In [ ]:
cd ~/workspace/repo/time_series/

In [ ]:
!pip install -r requirement.txt

In [ ]:
!pip install matplotlib cbor2 pycardano

In [ ]:
"""Single-script demo to show how to access on on-chain datum."""
# pylint: disable=W0718, C0206

import json
import logging
import sys
from collections import OrderedDict
from dataclasses import dataclass
from datetime import datetime, timezone
from typing import Final

import cbor2
import numpy
import pycardano
from pycardano import Address, Network, OgmiosChainContext, UTxO

OGMIOS_URL: Final[str] = "ws://ogmios.preprod.orcfax.io:1337"

# smart contract
ADA_USD_ORACLE_ADDR: Final[
    str
] = "addr_test1wrtcecfy7np3sduzn99ffuv8qx2sa8v977l0xql8ca7lgkgmktuc0"

auth_addr = Address.from_primitive(
    "addr_test1vrc7lrdcsz08vxuj4278aeyn4g82salal76l54gr6rw4ync86tfse"
)

# policy ID for the Auth tokens
AUTH_POLICY: Final[str] = "104d51dd927761bf5d50d32e1ede4b2cff477d475fe32f4f780a4b21"

network = Network.TESTNET
context = OgmiosChainContext(ws_url=OGMIOS_URL, network=network)

logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)-15s %(levelname)s :: %(filename)s:%(lineno)s:%(funcName)s() :: %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ",
    level="INFO",
)


def _decode_number(value_pair: list):
    """Decode a number value."""
    significand = numpy.uint64(value_pair[0]).astype(numpy.int64)
    base10_component = numpy.uint64(value_pair[1]).astype(numpy.int64)
    value = significand * 10 ** numpy.float_(base10_component)
    return value


def timestamp_to_human(timestamp):
    return datetime.utcfromtimestamp(int(timestamp) / 1000).strftime(
        "%Y-%m-%dT%H:%M:%SZ"
    )


def decode_utxo(utxo: pycardano.transaction.UTxO):
    """Split a UTxO into the components that we need to process and
    initially return the Orcfax Datum."""
    oracle_datum = cbor2.loads(utxo.output.datum.cbor)
    timestamp = oracle_datum.value[2].value[0]
    timestamp_human = timestamp_to_human(timestamp)
    logger.info("oracle datum timestamp: %s (%s)", timestamp_human, timestamp)
    labels = oracle_datum.value[0][b"name"].decode().split("|", 1)
    ada_usd = oracle_datum.value[0][b"value"][0].value
    pretty_log_value(ada_usd, labels[0])
    return (timestamp, _decode_number(ada_usd))


def pretty_log_value(value_pair: cbor2.CBORTag, label: str):
    """Return pretty logging information about a value pair."""
    value = _decode_number(value_pair)
    logger.info("%s: %s", label, value)


def validate_utxo(utxo: UTxO):
    """check if the token included in the utxo is the correct one."""
    valid = False
    for item in utxo.output.amount.multi_asset:
        if str(item) == AUTH_POLICY:
            valid = True
            for asset in utxo.output.amount.multi_asset[item]:
                amount = utxo.output.amount.multi_asset[item][asset]
                logger.info("found %d %s", amount, str(asset))
    return valid


def get_utxos(oracle_addr: str):
    """return the Orcfax UTxOs."""
    oracle_utxos = context.utxos(oracle_addr)
    utxos = []
    logger.info("inspecting '%s' UTxOs", len(oracle_utxos))
    for utxo in oracle_utxos:
        if not utxo.output.script and utxo.output.datum and validate_utxo(utxo):
            utxos.append(utxo)
    return utxos


def read_datum():
    """Get the timestamps and prices from all validated Orcfax UTxOs."""
    logger.info("entering this script... ")
    logger.info("oracle smart contract: %s", ADA_USD_ORACLE_ADDR)
    utxos = get_utxos(ADA_USD_ORACLE_ADDR)
    times_prices = []
    if not utxos:
        logger.info("no oracle data found")
        sys.exit(0)
    for utxo in utxos:
        (timestamp, ada_usd) = decode_utxo(utxo)
        times_prices.append((timestamp, ada_usd))
    return times_prices

In [ ]:
times_prices = read_datum()
times_prices.sort(key=lambda tp: tp[0])

In [ ]:
import matplotlib.pyplot as plt
for tp in times_prices:
    print("{0},{1}".format(timestamp_to_human(tp[0]),tp[1]))

plt.plot(*zip(*times_prices))

In [ ]:
cd ~/workspace/repo/time_series

In [ ]:
# UNCOMMENT TO RUN THE SERVICE
# !python3 service.py

In [ ]:
# WARNING: UNCOMMENT AND RUN THE CODE BELOW IFF USING PYTHON>=3.10
# THIS IS MEANT TO MAKE THE SERVICE COMPATIBLE.
# def findReplace(filename, find, replace):
#     with open(filename, 'r') as file:
#         filedata = file.read()
#     filedata = filedata.replace(find, replace)
#     with open(filename, 'w') as file:
#         file.write(filedata)

# findReplace("/config/.local/lib/python3.10/site-packages/attrdict/default.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/attrdict/mapping.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/attrdict/merge.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/attrdict/mixins.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/eth_account/account.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/rlp/codec.py", "import collections\n", "import collections.abc as collections\n")
# findReplace("/config/.local/lib/python3.10/site-packages/rlp/lazy.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/rlp/sedes/lists.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/rlp/sedes/raw.py", "from collections import", "from collections.abc import")
# findReplace("/config/.local/lib/python3.10/site-packages/rlp/sedes/serializable.py", "import collections\n", "import collections\nimport collections.abc\n")
# findReplace("/config/.local/lib/python3.10/site-packages/rlp/sedes/serializable.py", "collections.Sequence", "collections.abc.Sequence")
# findReplace("/config/.local/lib/python3.10/site-packages/web3/datastructures.py", "from collections import (\n    Hashable,\n    Mapping,\n    MutableMapping,\n    OrderedDict,\n    Sequence,\n)", "from collections.abc import (\n    Hashable,\n    Mapping,\n    MutableMapping,\n    Sequence,\n)\nfrom collections import OrderedDict")
# findReplace("/config/.local/lib/python3.10/site-packages/web3/utils/six/six.py", "import collections\n", "import collections.abc as collections\n")
# findReplace("/config/.local/lib/python3.10/site-packages/web3/utils/formatters.py", "from collections import", "from collections.abc import")